## A Simple Approach To Templated SQL Queries In Python

In [2]:
#  install duckdb

In [28]:
import pandas as pd
import xlrd
import pyodbc
import warnings
warnings.simplefilter('ignore', UserWarning)
import sqlalchemy as sa
from datetime import datetime
import numpy as np
import duckdb
##%%
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [29]:
duckdb_conn = duckdb.connect(database=':memory:')

## query example

In [92]:
query="""
select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = 1234
    and transaction_date = '2019-03-02'
group by
    user_id
"""

In [93]:
## in jinja

In [94]:
j_query="""
select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = {{ uid }}
    and transaction_date = {{ tdate }}
group by
    user_id
    """

In [95]:
user_transaction_template = '''
select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = {{ user_id }}
    and transaction_date = {{ transaction_date }}
group by
    user_id
'''

In [96]:
## set params

In [103]:
params = {
    'user_id': 1234,
    'transaction_date': '2019-03-02',
}

In [104]:
## generate sql query

In [105]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='pyformat')
query, bind_params = j.prepare_query(user_transaction_template, params)

In [106]:
print(query)



select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = %(user_id_1)s
    and transaction_date = %(transaction_date_2)s
group by
    user_id


In [107]:
print(bind_params)

{'user_id_1': 1234, 'transaction_date_2': '2019-03-02'}


In [108]:
print(query % bind_params)


select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = 1234
    and transaction_date = 2019-03-02
group by
    user_id


In [44]:
## example to read

In [45]:

# import pandas as pd
# frm = pd.read_sql(query, conn, params=bind_params)

## from DB

In [120]:
duckdb_conn = duckdb.connect(database=':memory:')
duckdb_conn.execute("""
   ATTACH 'dbname=analytics user=analytic password=R6MRbe6tT4vS2n host=192.168.9.234 port=5432'
   AS pg (TYPE POSTGRES)""")

In [121]:
query="""
select * from pg.dwh.hr_structure limit 3
"""

In [122]:
duckdb_conn.execute(query).df()

,company,division,departaments,unit_,group_,amats
0,Polipaks,Production division/ Operational block,Technical department,None,None,Galvenais mehāniķis
1,Polipaks,Production division/ Operational block,Production department,PRINT_LAM,PRINT (1),1.iespiedmašīnas operators
2,Polipaks,Service departments,IT,None,None,1C speciāliste


In [123]:
user_transaction_template2 = '''
select * from pg.dwh.hr_structure
where departaments={{ dep }}
limit 3
'''

In [124]:
params = {
    'dep': 'IT',
    
}

In [125]:
from jinjasql import JinjaSql
j = JinjaSql(param_style='qmark')
query, bind_params = j.prepare_query(user_transaction_template2, params)

In [131]:
query,( bind_params)

('\nselect * from pg.dwh.hr_structure\nwhere departaments=?\nlimit 3', ('IT',))

In [126]:
frm = duckdb_conn.execute(query, parameters=bind_params).df()

In [127]:
print(bind_params)

('IT',)


In [128]:
print(query)


select * from pg.dwh.hr_structure
where departaments=?
limit 3


In [89]:
frm

,company,division,departaments,unit_,group_,amats
0,Polipaks,Service departments,IT,None,None,1C speciāliste
1,Polipaks,Service departments,IT,None,None,ERP administrators un atbalsta speciālists
2,Polipaks,Service departments,IT,None,None,ERP ieviešanas un atbalsta speciālists


## From a template to the final SQL query

In [109]:
from six import string_types
def quote_sql_string(value):
    '''
    If `value` is a string type, escapes single quotes in the string
    and returns the string enclosed in single quotes.
    '''
    if isinstance(value, string_types):
        new_value = str(value)
        new_value = new_value.replace("'", "''")
        return "'{}'".format(new_value)
    return value

In [110]:
## test

In [112]:
val=1
vls='1'

In [113]:
quote_sql_string(val)

1

In [114]:
quote_sql_string(vls)

"'1'"

In [115]:
from copy import deepcopy
def get_sql_from_template(query, bind_params):
    if not bind_params:
        return query
    params = deepcopy(bind_params)
    for key, val in params.items():
        params[key] = quote_sql_string(val)
    return query % params

In [116]:
print(get_sql_from_template(query, bind_params))


select
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
from
    transactions
where
    user_id = 1234
    and transaction_date = '2019-03-02'
group by
    user_id


In [119]:
query , bind_params

('\nselect\n    user_id\n    , count(*) as num_transactions\n    , sum(amount) as total_amount\nfrom\n    transactions\nwhere\n    user_id = %(user_id_1)s\n    and transaction_date = %(transaction_date_2)s\ngroup by\n    user_id',
 {'user_id_1': 1234, 'transaction_date_2': '2019-03-02'})